# GIS module

Les opérations GIS font partie intégrantes des opérations réalisées en hydrologie. Cette page 

In [ ]:
import geopandas as gpd
import pandas as pd
import warnings
import leafmap
import numpy as np
import xarray as xr


from pathlib import Path

import xdatasets as xd
import xhydro.gis as xhgis
from xhydro.indicators import get_yearly_op

## Creating a map

In [ ]:
m = leafmap.Map(center=(48.63, -74.71), 
                zoom=3,
                basemap="USGS Hydrography")
m

## Watershed delineation

### a) From a list of coordinates

In [ ]:
lng_lat = [(-69.81971, 48.14720), # Lac Saint-Jean watershed
           (-74.393438, 45.572442) # Ottawa river watershed
          ]

### b) From markers on a map

![test](../../docs/_static/_images/example_draw_marker,png)

In [ ]:
# The following code is only useful for the documentation. You should instead remove this code and
# interact with the map as shown above by positionning markers at sites of interest
m.draw_features = [{'type': 'Feature',
  'properties': {},
  'geometry': {'type': 'Point', 'coordinates': [-73.118597, 46.042467]}}] # Richelieu watershed

In [ ]:
%%time

gdf = xhgis.watershed_delineation(coordinates=lng_lat,
                                  map=m)
gdf

In [ ]:
m.zoom_to_gdf(gdf)

### c) From [xdatasets](https://github.com/hydrologie/xdatasets) (Not implemented yet)

This functionality fetches a list of basins from [xdatasets](https://github.com/hydrologie/xdatasets)' supported datasets, and upon request, [xdatasets](https://github.com/hydrologie/xdatasets) provides a `gpd.GeoDataFrame` containing the precalculated boundaries for these basins.

## Extract geographical watershed properties

a) Let's first extract watershed properties

In [ ]:
xhgis.watershed_properties(gdf)

In [ ]:
xhgis.watershed_properties(gdf[['HYBAS_ID', 'geometry']],
                           unique_id='HYBAS_ID',
                           output_format='xarray')

b) Let's then extract climate indicators

In [ ]:
# This will be returned by xdatasets ( c) above) eventually
bucket = Path("https://s3.us-east-2.wasabisys.com/watersheds-polygons/MELCC/json")

paths = [
    bucket.joinpath("023003/023003.json"),
    bucket.joinpath("031101/031101.json"),
]

gdf = pd.concat([gpd.read_file(path) for path in paths]).reset_index(drop=True)
gdf

In [ ]:
datasets = {
    "era5_land_reanalysis": {"variables": ["t2m", "tp", "sd"]},
}
space = {
    "clip": "polygon",  # bbox, point or polygon
    "averaging": True,
    "geometry": gdf,  # 3 polygons
    "unique_id": "Station",
}
time = {
    "start": "1981-01-01",
    "end": "2010-12-31",
    "timezone": "America/Montreal",
}

xds = xd.Query(datasets=datasets, 
               space=space, 
               time=time)



In [ ]:
# This should be what xdatasets returns so users don't have to add it manually
# We should also consider using xarray-pint to improve units conversion 

ds = xds.data.squeeze()
ds["Station"].attrs["cf_role"] = "timeseries_id"
ds['tas'] = ds.t2m - 273.15
ds["tas"].attrs = {"long_name": "2 metre temperature", "units": "C", "standard_name": "air_temperature", "cell_methods": "time: mean"}
ds["tp"].attrs = {"long_name": "Mean daily precipitation flux", "units": "kg m-2 s-1",
                  "standard_name": "precipitation_flux", "cell_methods": "time: mean within days"}

ds["sd"].attrs = {"long_name": "Snow depth", "units": "m",
                  "standard_name": "lwe_thickness_of_surface_snow_amount", "cell_methods": "time: mean within days"}
ds

In [ ]:
timeargs = {
    "01": {"month": [1]},
    "02": {"month": [2]},
    "03": {"month": [3]},
    "04": {"month": [4]},
    "05": {"month": [5]},
    "06": {"month": [6]},
    "07": {"month": [7]},
    "08": {"month": [8]},
    "09": {"month": [9]},
    "10": {"month": [10]},
    "11": {"month": [11]},
    "12": {"month": [12]},
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer_fall": {"date_bounds": ["06-20", "11-19"]},
    "year": {"date_bounds": ["01-01", "12-31"]},
    }

operations = {
    "tas": ["max","mean", "min"],
    "tp": ["sum"], 
    "sd": ["mean"], 
}


In [ ]:
ds_climatology = xr.merge(
    [
        get_yearly_op(ds, input_var=variable, op=op, timeargs=timeargs)
        for (variable, ops) in operations.items() for op in ops
    ]
)
ds_climatology

In [ ]:
pd.set_option('display.max_rows', 100)
ds_climatology.mean('time').to_dataframe().T